In [1]:
# Tool Calling Workflow (TCW) - A tool for managing and executing workflows
#1) Tool Definition
#2) Tool Binding
#3) Tool Execution
#4) The llm uses the tool to execute the workflow

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
llm.invoke("What is the weather in Trivandrum?")

AIMessage(content="I don't have real-time weather data. To get the current weather in Trivandrum, I recommend checking a reliable weather website or app for the most up-to-date information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 17, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwAH8psar6BKExQBIHAHgt72UW1gi', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--19534388-3afd-4b05-8e28-526a47cab477-0', usage_metadata={'input_tokens': 17, 'output_tokens': 36, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.tools import tool

In [5]:
@tool
def get_weather(location: str) -> str:
    """
    A mock weather API that returns the weather for a given location.
    """
    return f"The weather in {location} is sunny with a high of 30°C."

In [6]:
get_weather.invoke({"location": "Trivandrum"})

'The weather in Trivandrum is sunny with a high of 30°C.'

In [7]:
get_weather

StructuredTool(name='get_weather', description='A mock weather API that returns the weather for a given location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002B64FD90360>)

In [8]:
tools = [get_weather]

In [9]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
result = llm_with_tools.invoke("What is the weather in Trivandrum?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Sq5S2P8jAyzsIMdUBw9dVgxH', 'function': {'arguments': '{"location":"Trivandrum"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwAHAGfn6RF1U7gyee96kRFNWd2Ii', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3dc0ed81-3921-49bb-bcb6-dee1bca31098-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Trivandrum'}, 'id': 'call_Sq5S2P8jAyzsIMdUBw9dVgxH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {

In [11]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Trivandrum'},
  'id': 'call_Sq5S2P8jAyzsIMdUBw9dVgxH',
  'type': 'tool_call'}]

In [12]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(content="What is the weather in Trivandrum?")]

llm_output = llm_with_tools.invoke(messages)
llm_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_a1E76JjiE309ZmgvvSXh5fsy', 'function': {'arguments': '{"location":"Trivandrum"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwAHCSm5ffz09yThJ908sSvHBYsDo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bc458982-74d0-4726-ae2d-cbcbb599a9c5-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Trivandrum'}, 'id': 'call_a1E76JjiE309ZmgvvSXh5fsy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {

In [25]:
tool_mapping = {"get_weather": get_weather}

In [18]:
for tool_call in result.tool_calls:
    tool = tool_mapping[tool_call['name'].lower()]
    tool_output = tool.invoke(tool_call['args'])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call['id']))

In [19]:
messages

[HumanMessage(content='What is the weather in Trivandrum?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='The weather in Trivandrum is sunny with a high of 30°C.', tool_call_id='call_Sq5S2P8jAyzsIMdUBw9dVgxH')]

In [20]:
tool_call

{'name': 'get_weather',
 'args': {'location': 'Trivandrum'},
 'id': 'call_Sq5S2P8jAyzsIMdUBw9dVgxH',
 'type': 'tool_call'}

In [22]:
tool_call['name'].lower()

'get_weather'

In [26]:
tool_mapping[tool_call['name'].lower()]

StructuredTool(name='get_weather', description='A mock weather API that returns the weather for a given location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002B64FD90360>)

In [27]:
get_weather

StructuredTool(name='get_weather', description='A mock weather API that returns the weather for a given location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002B64FD90360>)

In [28]:
messages

[HumanMessage(content='What is the weather in Trivandrum?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='The weather in Trivandrum is sunny with a high of 30°C.', tool_call_id='call_Sq5S2P8jAyzsIMdUBw9dVgxH')]